In [1]:
import pandas as pd
from em_utilities import *
import sframe as sf
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
dataset= sf.SFrame('KO_articles.csv')
dataset.remove_column('X1')

[INFO] sframe.cython.cy_server: SFrame v2.1 started. Logging /tmp/sframe_server_1503245712.log


------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/abdl-rahman/Desktop/Recommendation systems/KO_articles.csv

Parsing completed. Parsed 1390 lines in 0.264099 secs.

category,text
business,Policy for Growth andInnovationI get asked ...
business,Sam AltmanThe mostimportant story of 2014 ...
business,Bubble talkI m tired ofreading about investors ...
business,A new team at redditLastweek Yishan Wong ...
business,Why Ops Is Taking OverStartup LandA little ...
business,10 Data AcquisitionStrategies for Startups ...
business,One of the GreatestEntrepreneurial Stories ...
business,2017 YC Annual LetterDearYC Community In ...
business,How to Build a StartupEcosystem in Your ...
business,Growth vs Profitabilityand Venture ReturnsThere ...


In [3]:
tfidfvec= TfidfVectorizer()
tf_idf_matrix= tfidfvec.fit_transform(dataset['text'])

In [4]:
tf_idf_matrix = normalize(tf_idf_matrix)

In [5]:
for i in range(5):
    doc = tf_idf_matrix[i]
    print(np.linalg.norm(doc.todense()))

1.0
1.0
1.0
1.0
1.0


In [6]:
#Smart Initialization for means
from sklearn.cluster import KMeans
np.random.seed(5)
num_clusters = 3

# Use scikit-learn's k-means to simplify workflow
#kmeans_model = KMeans(n_clusters=num_clusters, n_init=5, max_iter=400, random_state=1, n_jobs=-1) # uncomment to use parallelism -- may break on your installation
kmeans_model = KMeans(n_clusters=num_clusters, n_init=5, max_iter=400, random_state=1, n_jobs=1)
kmeans_model.fit(tf_idf_matrix)
centroids, cluster_assignment = kmeans_model.cluster_centers_, kmeans_model.labels_

means = [centroid for centroid in centroids]

In [7]:
#Smart initialization for weights
num_docs = tf_idf_matrix.shape[0]
weights = []
for i in xrange(num_clusters):
    # Compute the number of data points assigned to cluster i:
    num_assigned = len(cluster_assignment[cluster_assignment==i]) # YOUR CODE HERE
    w = float(num_assigned) / num_docs
    weights.append(w)

In [8]:
covs = []
for i in xrange(num_clusters):
    member_rows = tf_idf_matrix[cluster_assignment==i]
    cov = (member_rows.multiply(member_rows) - 2*member_rows.dot(diag(means[i]))).sum(axis=0).A1 / member_rows.shape[0] \
          + means[i]**2
    cov[cov < 1e-8] = 1e-8
    covs.append(cov)

In [9]:
model_em = EM_for_high_dimension(tf_idf_matrix, means, covs, weights, cov_smoothing=1e-10)

In [36]:
# Fill in the blanks
def visualize_EM_clusters(tf_idf, means, covs, map_index_to_word):
    print('')
    print('==========================================================')

    num_clusters = len(means)
    for c in xrange(num_clusters):
        print('Cluster {0:d}: Largest mean parameters in cluster '.format(c))
        print('\n{0: <12}{1: <12}{2: <12}'.format('Word', 'Mean', 'Variance'))
        
        # The k'th element of sorted_word_ids should be the index of the word 
        # that has the k'th-largest value in the cluster mean. Hint: Use np.argsort().
        sorted_word_ids = np.argsort(means[c])[::-1]  # YOUR CODE HERE

        for i in sorted_word_ids[:10]:
            print '{0: <12}{1:<10.2e}{2:10.2e}'.format(map_index_to_word[i], 
                                                       means[c][i],
                                                       covs[c][i])
        print '\n=========================================================='

In [37]:
visualize_EM_clusters(tf_idf_matrix, model_em['means'], model_em['covs'], tfidfvec.get_feature_names())


Cluster 0: Largest mean parameters in cluster 

Word        Mean        Variance    
netflix     6.42e-01    3.54e-03
blog        2.59e-01    4.02e-03
technology  1.83e-01    2.02e-03
regarding   1.69e-01    1.73e-03
perspectives1.65e-01    1.64e-03
the         1.54e-01    3.62e-04
tech        1.42e-01    2.25e-03
and         1.38e-01    3.71e-04
issues      1.06e-01    2.73e-04
challenges  1.04e-01    6.58e-04

Cluster 1: Largest mean parameters in cluster 

Word        Mean        Variance    
the         2.17e-01    8.76e-03
to          1.47e-01    3.63e-03
and         1.26e-01    3.44e-03
of          1.12e-01    2.72e-03
in          7.65e-02    1.59e-03
is          6.48e-02    1.46e-03
that        5.77e-02    1.47e-03
for         4.90e-02    8.96e-04
we          4.81e-02    3.25e-03
it          4.54e-02    1.19e-03

Cluster 2: Largest mean parameters in cluster 

Word        Mean        Variance    
the         2.95e-01    5.99e-03
to          2.53e-01    3.03e-03
and         1.84